# Kaggle Project
## Linear Model

In [1]:
# Import required libraries
from __future__ import division
import numpy as np
import mltools as ml
import matplotlib.pyplot as plt

np.random.seed(0)
%matplotlib inline

In [2]:
path_to_file = "/Users/justinchung/Documents/School/CS178/HW4/data"

# Training data
X = np.genfromtxt(path_to_file + '/X_train.txt', delimiter=None)
Y = np.genfromtxt(path_to_file + '/Y_train.txt', delimiter=None)
X,Y = ml.shuffleData(X,Y)

Xtr,Xva,Ytr,Yva = ml.splitData(X,Y); # split data into 80/20 train/validation
Xtr, Ytr = ml.shuffleData(Xtr, Ytr)
#Xtr, Ytr = Xtr[:8000], Ytr[:8000]

# Test data
Xte = np.genfromtxt(path_to_file + '/X_test.txt', delimiter=None)

In [3]:
# Rescale training and test data
Xtr,params = ml.transforms.rescale(Xtr)
Xva,_ = ml.transforms.rescale(Xva, params)

In [4]:
XtrP = ml.transforms.fpoly(Xtr, 2, bias=False)
XvaP = ml.transforms.fpoly(Xva, 2, bias=False)

XtrP,paramsP = ml.transforms.rescale(XtrP)
XvaP,_ = ml.transforms.rescale(XvaP, paramsP)

XteP = ml.transforms.fpoly(Xte, 2, bias=False)
XteP,_ = ml.transforms.rescale(XteP)

In [5]:
print(XtrP.shape)
print(XvaP.shape)
print(XteP.shape)

(160000, 119)
(40000, 119)
(200000, 119)


In [6]:
learner = ml.linearC.linearClassify()
learner.train(XtrP, Ytr, reg=0.1, initStep=0.01, stopTol=1e-6, stopIter=100)

probs = learner.predictSoft(XteP)
print(probs)
print("Train AUC: " + str(learner.auc(XtrP, Ytr)))
print("Validation AUC: " + str(learner.auc(XvaP, Yva)))

[[ 0.65037406  0.34962594]
 [ 0.5861603   0.4138397 ]
 [ 0.64398876  0.35601124]
 ..., 
 [ 0.68422899  0.31577101]
 [ 0.64840227  0.35159773]
 [ 0.66142157  0.33857843]]
0.658588582897
0.654150604309


In [11]:
Yte = np.vstack((np.arange(XteP.shape[0]), learner.predictSoft(XteP)[:,1])).T

# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('Y_submit.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')

In [34]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

depths = [5]
# 5 - 8
for depth in depths:
    dtree = DecisionTreeClassifier(max_depth=depth)

    abc = AdaBoostClassifier(dtree, n_estimators=300)
    abc.fit(Xtr, Ytr)

    predictions_Xtr = abc.predict(Xtr)
    predictions_Xva = abc.predict(Xva)
    
    probs_Xva = abc.staged_predict_proba(Xva)
    probs_Xte = abc.predict_proba(Xte)
    print(depth)
    print("Train AUC: " + str(metrics.roc_auc_score(predictions_Xtr, Ytr)))
    print("Validation AUC: " + str(metrics.roc_auc_score(predictions_Xva, Yva)))

5
Train AUC: 0.789297171955
Validation AUC: 0.691640738819


In [29]:
probs_Xva = abc.staged_predict_proba(Xva)
probs_Xte = abc.predict_proba(Xte)

In [30]:
print("Train AUC: " + str(metrics.roc_auc_score(predictions_Xtr, Ytr)))
print("Validation AUC: " + str(metrics.roc_auc_score(predictions_Xva, Yva)))

Train AUC: 0.663693108006
Validation AUC: 0.659220188078
